<a href="https://colab.research.google.com/github/riazzzz/softcomp/blob/main/rnn_assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


Saving dataset1.csv to dataset1.csv


In [148]:

import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D ,Bidirectional, SimpleRNN
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re



Only keeping the necessary columns.

In [149]:
data = pd.read_csv('dataset1.csv', encoding = "ISO-8859-1", engine='python')

data.head()


,message,label
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [150]:
data = data[['message','label']]

In [151]:
data = data[data.label != "Neutral"]
data['message'] = data['message'].apply(lambda x: x.lower())
data['message'] = data['message'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]','',x)))


print(data[ data['label'] == 0].size)
print(data[ data['label'] == 1].size)


data.head()


16000
4628


,message,label
0,just had a real good moment i missssssssss him...,0
1,is reading manga httpplurkcompmzp1e,0
2,comeagainjen httptwitpiccom2y2lx httpwwwyoutu...,0
3,lapcat need to send em to my accountant tomorr...,0
4,add me on myspace myspacecomlookthunder,0


In [152]:

import nltk
nltk.download('stopwords')
  
stopwords=nltk.corpus.stopwords.words('english')

def remove_stopwords(texts):
  txt_clean=" ".join([word for word in texts.split() if word not in stopwords])
  return txt_clean
data['message'] = data['message'].apply(lambda x: remove_stopwords(x))
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,message,label
0,real good moment missssssssss much,0
1,reading manga httpplurkcompmzp1e,0
2,comeagainjen httptwitpiccom2y2lx httpwwwyoutub...,0
3,lapcat need send em accountant tomorrow oddly ...,0
4,add myspace myspacecomlookthunder,0


In [153]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['message'].values)

X = tokenizer.texts_to_sequences(data['message'].values)
print(X)
X = pad_sequences(X)
print(X.shape[1])


[[113, 3, 563, 6352, 27], [388, 6353, 6354], [3110, 6355, 6356], [6357, 43, 461, 761, 6358, 64, 6359, 377, 75, 1857, 6360, 1481, 1208, 104], [312, 586, 6361], [961, 3, 213, 96, 104], [6362, 6363, 84, 150, 174, 6364, 61, 238, 210, 1332], [1333, 6365, 1121, 18, 62], [587, 3111, 1122, 6366], [6367, 28, 6368, 230, 762], [3, 41, 633], [138, 1209, 2564, 271, 258, 150, 271, 1334, 295, 1646, 6369], [6370, 2565, 7, 16, 696, 7, 4100, 16, 304, 118, 462], [3112, 83, 304, 547, 71, 79, 529], [25, 5, 6371, 2566], [6372, 2154, 4101, 6, 4101, 4102, 119], [6373, 2155, 2567, 139, 271, 6374, 35, 7, 313, 547, 2156], [6375, 6376, 2568, 6377], [2569, 333, 178, 206, 3, 93, 6, 175], [6378, 154, 22, 175, 661, 38, 727], [288, 463, 106, 26, 35, 120, 1335, 4103, 425], [728, 14, 4104, 6379, 344, 6380, 6381, 33, 1210, 181, 19, 6382, 121], [6383, 81, 85, 1123, 76, 1336, 4105, 219], [6384, 125, 2570, 45, 246, 63, 4106, 2570, 9], [6385, 353, 6386, 51, 503, 3113, 6387, 6388, 6389], [728, 333, 33, 283, 2157, 613], [136, 

In [154]:

embed_dim = 128
RNN_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(SimpleRNN(RNN_out,return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(SimpleRNN(20, dropout=0.5,return_sequences=True))
model.add(SimpleRNN(10, dropout=0.5))


model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 43, 128)           1280000   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 43, 128)           0         
_________________________________________________________________
simple_rnn_21 (SimpleRNN)    (None, 43, 196)           63700     
_________________________________________________________________
simple_rnn_22 (SimpleRNN)    (None, 43, 20)            4340      
_________________________________________________________________
simple_rnn_23 (SimpleRNN)    (None, 10)                310       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 22        
Total params: 1,348,372
Trainable params: 1,348,372
Non-trainable params: 0
____________________________________________

In [155]:
Y = pd.get_dummies(data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6910, 43) (6910, 2)
(3404, 43) (3404, 2)


In [156]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)

Epoch 1/10
216/216 [==============================] - 23s 89ms/step - loss: 0.5814 - accuracy: 0.7425
Epoch 2/10
216/216 [==============================] - 18s 85ms/step - loss: 0.5448 - accuracy: 0.7722
Epoch 3/10
216/216 [==============================] - 19s 86ms/step - loss: 0.5833 - accuracy: 0.7239
Epoch 4/10
216/216 [==============================] - 19s 87ms/step - loss: 0.2786 - accuracy: 0.9007
Epoch 5/10
216/216 [==============================] - 19s 86ms/step - loss: 0.0953 - accuracy: 0.9835
Epoch 6/10
216/216 [==============================] - 19s 86ms/step - loss: 0.0765 - accuracy: 0.9870
Epoch 7/10
216/216 [==============================] - 19s 87ms/step - loss: 0.0890 - accuracy: 0.9822
Epoch 8/10
216/216 [==============================] - 19s 88ms/step - loss: 0.0670 - accuracy: 0.9878
Epoch 9/10
216/216 [==============================] - 19s 86ms/step - loss: 0.0831 - accuracy: 0.9841
Epoch 10/10
216/216 [==============================] - 19s 90ms/step - loss: 0.071

Extracting a validation set, and measuring score and accuracy.

In [157]:

score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

107/107 [==============================] - 2s 16ms/step - loss: 0.0670 - accuracy: 0.9877
score: 0.07
acc: 0.99


In [160]:
t = [' he is  in depression like all of us.']

t = tokenizer.texts_to_sequences(t)

t = pad_sequences(t, maxlen=X.shape[1], dtype='int32', value=0)
print(t)
sentiment = model.predict(t,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 1):
    print("negative")
elif (np.argmax(sentiment) == 0):
    print("positive")

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  6 68]]
1/1 - 0s
negative
